# Problem 1

Setting up the enrivonment and loading the data. No need to change this part.

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Load tensors from the file
loaded_tensors = torch.load('exercise_2_problem_1_data.pth')
X_tensor = loaded_tensors['X_tensor']
Y_tensor = loaded_tensors['Y_tensor']
dataset = TensorDataset(X_tensor, Y_tensor)

## a) Model definition

Finalize the model definition as instructed in the exercise sheet.

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            # TODO
        )

    def forward(self, x):
        return self.layers(x)


## b) Write optimization loop

You can use any optimizer you want, but remember to set all the hyperparameters it requires, including the batch size and number of iterations etc that are defined outside the optimizer function.

In [ ]:
# Helper for retrieving minibatches of desired size
batch_size = # You choose this!
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


model = RegressionModel(100, 2)
criterion = nn.MSELoss()
lr = # TODO
optimizer = # TODO

num_epochs = # TODO
printing_interval = num_epochs // 10 # Avoid printing hundreds or thousands of losses, to keep the notebook cleaner
losses_ = []

model.train()
# TODO: Define loop over epochs 
    running_loss = 0.0

    # TODO: Define loop over batches
    
        # 1) TODO: Zero the gradients
        # 2) TODO: Forward pass
        # 3) TODO: Compute loss (MSE)
        # 4) TODO: Backprop
        # 5) TODO: Update parameters

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    losses_.append(avg_loss)

    if(epoch % printing_interval == 1):
        print(f"Epoch [{epoch+1}/{num_epochs}] - MSE Loss: {avg_loss:.6f}")


## d) Computing the gradient norms

Practice extracting information about a trained model, by computing the gradient norms that were used to analyse SGD behavior during the lectures. No need to do anything with the gradient norms, just show that you can compute them.

In [ ]:
# Switch to evaluation mode -- we are no longer training!
model.eval()

grad_norms = []  # to store gradient norms for each batch

# TODO: Define loop over batches
    # Compute squared L2 norm across all model parameter gradients
    # Hints:
    # - You again need to first evaluate the gradient of the loss wrt to the model parameters
    # - You can extract a list of all model parameters with model.parameters()
    # - For each parameter p in model.parameters(), you can access the gradient with p.grad
    grad_norms.append(norm_for_this_batch)

# Convert to numpy array for statistics
grad_norms = np.array(grad_norms)
mean_gn = # TODO
var_gn  = # TODO

print("Gradient Norm stats for evaluation pass:")
print(f"  Mean  : {mean_gn:.6f}")
print(f"  Std   : {var_gn:.6f}")

## e) Reporting

**Edit this cell directly to write your answers.** You should print the values within the code blocks, but also copy them here for ease of grading. You can also write any other remarks you may have in this cell.

### Optimizer settings: 

TODO: I used ... with parameters ...

### Optimization speed:

TODO: I reached the loss threshold after ... epochs

### The gradient norms:

TODO: After training the model, the gradient norm is ... and the variance over the batches is ...

In [ ]:
# Plot the loss as function of epochs, using log-scale for the y-axis

=========

# Problem 2

## a) Generate training/testing data

Do not change this part. The code snippet greates synthetic data by feeding random inputs through randomly initialized neural network.

In [ ]:
# N samples with D inputs and O outputs
N = 100
D = 10
O = 5

# Random mapping from x to y, as small neural network
class CreationModel(nn.Module):
  def __init__(self, D, O, M):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(D, M),
      nn.Tanh(),
      nn.Linear(M, O)
    )

  def forward(self, x):
    return self.layers(x)

# For getting the same data
torch.manual_seed(78798)

# Training data
x = torch.randn(N, D)
noiselevel = 0.1
modelGenerate = CreationModel(D,O,5)
modelGenerate.eval()
y = modelGenerate(x).clone().detach() + noiselevel*torch.randn(N, O)

# Test data
N_test = 10000
x_test = torch.randn(N_test, D)
y_test = modelGenerate(x_test).clone().detach()  + noiselevel*torch.randn(N_test, O)

## Pre-defined model

Do not change this part. The parameter $M$ controls the size/complexity of the network.

In [ ]:
class RegressionModel(nn.Module):
  def __init__(self, D, O, M):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(D, M),
      nn.ReLU(),
      nn.Linear(M, M),
      nn.ReLU(),
      nn.Linear(M, O)
    )

  def forward(self, x):
    return self.layers(x)

## b) Train model and validate the double descent principle

In [ ]:
# Use full data for gradients
B = N
data_loader = DataLoader(TensorDataset(x,y), batch_size=B, shuffle=True)

# Loop over some range of M values. With these parameters logspace generates M=2 twice, so dropping the first entry
Mvalues = np.logspace(np.log10(2),np.log10(60),num=15,dtype='int')[1:]
losses = np.zeros((len(Mvalues),2))
for M_index, M in enumerate(Mvalues):
    # TODO: Define the model
    # TODO: Train it until convergence
    # TODO: Evaluate both training and test error, and store them

    losses[mi,0] = trainloss.item()
    losses[mi,1] = testloss.item()

### Plot the final training loss and the final test loss

The code below is provided as an example; you may need to modify it to be compatible with your results. Make some effort to make the plot easy to read.

In [ ]:
plt.loglog(Mvalues, losses[:,0], 'b-')
plt.loglog(Mvalues, losses[:,1], 'b:')
plt.legend(['Training loss', 'Test loss'])
ax = plt.gca(); ax.set_ylim([10**(-5),10**(0)])
ax.set_ylabel("MSE")
ax.set_xlabel("Number of neurons")
# Customize x-axis to show more detailed labels
from matplotlib.ticker import LogLocator, ScalarFormatter
ax.xaxis.set_major_locator(LogLocator(base=10.0, subs=np.arange(1, 10)))
ax.xaxis.set_major_formatter(ScalarFormatter())
plt.show()

### c) Having more data can hurt

Show that increasing the amount of data helps when around the interpolation threshold, but that for overparameterized model it may hurt.

In [ ]:
# Generate the larger training set, using the same process as before
N_large = 500
x_large = torch.randn(N_large, D)
y_large = modelGenerate(x_large).clone().detach() + noiselevel*torch.randn(N_large, O)

# TODO: Identify two values of M you need to answer the question

# TODO: Repeat the training process for those options. Note that you can use the same code as before, 
# since you only changed the data and the set of M

print("Test losses")
# TODO

## Reporting

**Edit this cell directly to report the requested information.**

### Interpolation threshold

TODO: The interpolation threshold is around M = ...

TODO: For that M, the model has in total ... parameters

TODO: Explain the findings. Do you observe the expected pattern? If not, what might have been a problem?

### More data can hurt

TODO: Explain your choices of M values, report the test errors for the different amounts of training data, and explain the findings.